In [1]:
import pandas as pd
pd.set_option("display.max_row", None)
%load_ext autoreload
%autoreload 2

In [6]:
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)
#localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

✅Connected, local lib:0.20.2, server:0.24.0

In [311]:
deal_data = {
    "name":"Yield Supplement Overcollateralization"
    ,"dates":{"cutoff":"2021-06-01"
              ,"closing":"2021-07-15"
              ,"firstPay":"2021-08-26"
              ,"payFreq":["DayOfMonth",20]
              ,"poolFreq":"MonthEnd"
              ,"stated":"2030-01-01"}
    ,"pool":{'assets':[["Mortgage"
                        ,{"originBalance":1300,"originRate":["fix",0.045],"originTerm":30
                          ,"freq":"Monthly","type":"Level","originDate":"2021-03-05"}
                          ,{"currentBalance":1300
                          ,"currentRate":0.08
                          ,"remainTerm":30
                          ,"status":"current"}]]}
    ,"accounts":{"acc01":{"balance":100}
                ,"acc02":{"balance":0}}
    ,"bonds":{"A1":{"balance":1000
                 ,"rate":0.07
                 ,"originBalance":1000
                 ,"originRate":0.07
                 ,"startDate":"2020-01-03"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2020-01-03"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{}
    ,"collect":[["CollectedCash","acc01"]]
    ,"waterfall":{"Amortizing":[
          ["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]
           ,{"limit":
             {"formula":("-",("schedulePoolValuation",('PvRate',("poolWaRate",)))
                            ,("schedulePoolValuation",('PvRate',("max",("poolWaRate",),("const",0.09)))))
             }}]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,"status":("PreClosing","Amortizing")
}

In [312]:
from absbox import mkDeal
deal = mkDeal(deal_data)  ## now a generic class created

In [313]:
r = localAPI.run(deal
                 ,poolAssump = None
                 ,runAssump = [("inspect",(["DayOfMonth",20],("schedulePoolValuation",('PvRate',0.08)))
                                          ,(["DayOfMonth",20],("schedulePoolValuation",('PvRate',("max",("poolWaRate",),("const",0.09)))))
                                          ,(["DayOfMonth",20],("-",("schedulePoolValuation",('PvRate',("poolWaRate",)))
                                                             ,("schedulePoolValuation",('PvRate',("max",("poolWaRate",),("const",0.09))))))
                                
                                )
                              ]
                 ,read=True)

In [314]:
from absbox.local.util import unifyTs

In [315]:
unifyTs(r['result']['inspect'].values())

,"<PoolScheduleCfPv:{'tag': 'PvByRef', 'contents': {'tag': 'Max', 'contents': [{'tag': 'PoolWaRate', 'contents': None}, {'tag': 'Constant', 'contents': {'numerator': 9, 'denominator': 100}}]}},None>","<PoolScheduleCfPv:{'tag': 'PvRate', 'contents': {'numerator': 2, 'denominator': 25}},None>","<Substract:{'tag': 'PoolScheduleCfPv', 'contents': [{'tag': 'PvByRef', 'contents': {'tag': 'PoolWaRate', 'contents': None}}, None]},{'tag': 'PoolScheduleCfPv', 'contents': [{'tag': 'PvByRef', 'contents': {'tag': 'Max', 'contents': [{'tag': 'PoolWaRate', 'contents': None}, {'tag': 'Constant', 'contents': {'numerator': 9, 'denominator': 100}}]}}, None]}>"
Date,,,
2021-06-01,0.00,0.00,0.00
2021-06-20,0.00,0.00,0.00
2021-07-20,1136.82,1147.91,11.09
2021-08-20,1097.07,1107.33,10.26
2021-09-20,1057.01,1066.51,9.50
2021-10-20,1016.40,1025.17,8.77
2021-11-20,975.76,983.80,8.04
2021-12-20,934.57,941.95,7.38
2022-01-20,893.31,900.03,6.72


In [316]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-08-26,989.87,8.05,10.13,0.07,18.18,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,980.37,4.74,9.50,0.07,14.24,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,971.60,5.64,8.77,0.07,14.41,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,963.56,5.77,8.04,0.07,13.81,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,956.18,5.54,7.38,0.07,12.92,"[<PayInt:A1>, <PayPrin:A1>]"
2022-01-20,949.46,5.68,6.72,0.07,12.40,"[<PayInt:A1>, <PayPrin:A1>]"
2022-02-20,943.38,5.64,6.08,0.07,11.72,"[<PayInt:A1>, <PayPrin:A1>]"
2022-03-20,937.86,5.06,5.52,0.07,10.58,"[<PayInt:A1>, <PayPrin:A1>]"
2022-04-20,932.93,5.57,4.93,0.07,10.50,"[<PayInt:A1>, <PayPrin:A1>]"


In [317]:
r['accounts']['acc01']

,balance,change,memo
date,,,
2021-07-15,195.90,95.90,<Pool:CollectedCash>
2021-07-31,195.90,0.00,<Pool:CollectedCash>
2021-08-26,187.85,-8.05,<PayInt:A1>
2021-08-26,177.72,-10.13,"[<PayPrin:A1>, <DS:Substract [PoolScheduleCfPv..."
2021-08-26,0.00,-177.72,<PayPrin:B>
2021-08-26,0.00,0.00,<PayPrin:B>
2021-08-31,47.95,47.95,<Pool:CollectedCash>
2021-09-20,43.21,-4.74,<PayInt:A1>
2021-09-20,33.71,-9.50,"[<PayPrin:A1>, <DS:Substract [PoolScheduleCfPv..."
